In [1]:
import pandas as pd
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
n_samples=50000
n_features=20
X, y = make_classification(n_samples=n_samples, n_features=n_features, n_informative=5, n_redundant=2, n_repeated=0, n_classes=2,\
                           n_clusters_per_class=2, weights=[0.9,0.1], flip_y=0.01, class_sep=1.0, hypercube=True, shift=0.0,\
                           scale=1.0, shuffle=True, random_state=None)
y=np.array(y)
X=np.array(X)
y=np.reshape(y,(n_samples, 1))
X=np.reshape(X, (n_samples, n_features))
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.33, random_state=42)


In [16]:
y

array([[0],
       [0],
       [0],
       ...,
       [0],
       [0],
       [0]])

In [2]:
#custom scoring function sklearn
from sklearn.grid_search import GridSearchCV
from sklearn.metrics.scorer import make_scorer
from sklearn.metrics import roc_auc_score
import numpy as np
import pandas as pd
def lift_index(y, probabilities):
    #rearrange probabilities and y in descending order
    probabilities = [val[0] for val in probabilities]
    print(probabilities)
    combine = list(zip(y, probabilities))
    dtype = [('data', int), ('prob', float)]
    combine = np.array(combine,dtype=dtype)
    combine = np.sort(combine, order='prob') #sort according to probabilities
    ordered = np.sort(probabilities) #sort probabilities
    #divide into deciles
    deciles = pd.qcut(ordered, 10, duplicates='drop', labels=False) #creates array of digits from 1 to 10
    #sum of positive in each decile-if negative then zero and doesn't get taken into account
    sum_deciles = sum([((val+1)/10)*combine[num][0] for num, val in enumerate(deciles)])
    return sum_deciles/sum(y)
lift_index_score = make_scorer(lift_index, greater_is_better=True)

C:\Users\Snir\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Users\Snir\Anaconda3\lib\site-packages\sklearn\grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [20]:

from __future__ import print_function

import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.noise import AlphaDropout
from keras.utils import np_utils
from keras.optimizers import RMSprop
from keras.initializers import lecun_uniform
from keras.wrappers.scikit_learn import KerasClassifier
from keras import regularizers

def construct_model(layers=[50,50], activation='relu', b_init='random_uniform', k_init='random_uniform', dropout=.0,\
                    a_dropout=.0, optimizer='sgd', loss='binary_crossentropy', metrics='roc_auc_score',input_dim=20, output_dim=1, model_name='most'):
    """
    Helper to construct a Keras model based on dict of specs and input size

    Parameters
    ----------
    model_spec: dict
        Dict containing keys: arch, activation, dropout, optimizer, loss,
            w_reg, metrics
    input_dim: int
        Size of input dimension
    output_dim: int
        Size of input dimension

    Returns
    -------
    model: Compiled keras.models.Sequential

    """

    model = Sequential()

    for li, layer_size in enumerate(layers):
        # For input layer, add input dimension
        if li == 0:
            temp_input_dim = input_dim
            model.add(Dense(layer_size,
                            input_dim=temp_input_dim,
                            activation=activation,
                            bias_initializer=b_init,
                            kernel_initializer=k_init,
                            name='Input'))
        elif li < len(layers)-1:
            model.add(Dense(layer_size,
                            activation=activation,
                            bias_initializer=b_init,
                            kernel_initializer=k_init,
                            name='Layer_%i' % li))
        elif li == len(layers)-1:
            model.add(Dense( output_dim ,
                            activation='sigmoid',
                            bias_initializer=b_init,
                            kernel_initializer=k_init,
                            name='Layer_%i' % li))
            
        if dropout > 0. :
            if a_dropout == 0.:
                model.add(Dropout(dropout, name='Dropout_%i' % li))
        if a_dropout > 0.:
            model.add(Dropout(a_dropout, name='Dropout_%i' % li))
    model.compile(optimizer=optimizer,
                  loss=loss,
                  metrics=[metrics],
                 loss_weights={'0': 1,'1': 5})
    #keras.callbacks.TensorBoard(log_dir='./logs', histogram_freq=0, save_best_only=True, \
    #                                write_graph=True, write_images=False)

    return model 

In [21]:
model = construct_model(layers=[50,50], activation='selu', b_init='lecun_uniform', k_init='lecun_uniform', dropout=.0,\
                    a_dropout=.2, optimizer='sgd', loss='binary_crossentropy', metrics='accuracy',input_dim=20, output_dim=1, model_name='most')
model.fit(X_train, y_train, batch_size=32, epochs=100)
loss_and_metrics = model.evaluate(X_test, y_test, batch_size=128)

ValueError: Unknown entry in loss_weights dictionary: "0". Only expected the following keys: ['Dropout_1']

In [6]:
loss_and_metrics

[1.0825367202903284, 0.9287878788745765]

In [14]:
model.predict(X_test)

array([[0.0000000e+00],
       [0.0000000e+00],
       [0.0000000e+00],
       [0.0000000e+00],
       [0.0000000e+00],
       [0.0000000e+00],
       [0.0000000e+00],
       [0.0000000e+00],
       [0.0000000e+00],
       [0.0000000e+00],
       [1.0000000e+00],
       [0.0000000e+00],
       [0.0000000e+00],
       [0.0000000e+00],
       [0.0000000e+00],
       [0.0000000e+00],
       [0.0000000e+00],
       [0.0000000e+00],
       [0.0000000e+00],
       [7.2877667e-25],
       [0.0000000e+00],
       [0.0000000e+00],
       [0.0000000e+00],
       [0.0000000e+00],
       [0.0000000e+00],
       [2.2293985e-36],
       [0.0000000e+00],
       [0.0000000e+00],
       [0.0000000e+00],
       [0.0000000e+00]], dtype=float32)

stratified generates random predictions by respecting the training set class distribution.
most_frequent always predicts the most frequent label in the training set.
prior always predicts the class that maximizes the class prior (like most_frequent`) and ``predict_proba returns the class prior.
uniform generates predictions uniformly at random.
constant always predicts a constant label that is provided by the user.

In [ ]:
from sklearn.dummy import DummyClassifier 

clf = DummyClassifier(strategy='most_frequent',random_state=0)
clf.fit(X_train, y_train)
clf.score(X_test, y_test)  

In [ ]:
def plot_history(history):
    loss_list = [s for s in history.history.keys() if 'loss' in s and 'val' not in s]
    val_loss_list = [s for s in history.history.keys() if 'loss' in s and 'val' in s]
    acc_list = [s for s in history.history.keys() if 'acc' in s and 'val' not in s]
    val_acc_list = [s for s in history.history.keys() if 'acc' in s and 'val' in s]
    
    if len(loss_list) == 0:
        print('Loss is missing in history')
        return 
    
    ## As loss always exists
    epochs = range(1,len(history.history[loss_list[0]]) + 1)
    
    ## Loss
    plt.figure(1)
    for l in loss_list:
        plt.plot(epochs, history.history[l], 'b', label='Training loss (' + str(str(format(history.history[l][-1],'.5f'))+')'))
    for l in val_loss_list:
        plt.plot(epochs, history.history[l], 'g', label='Validation loss (' + str(str(format(history.history[l][-1],'.5f'))+')'))
    
    plt.title('Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    
    ## Accuracy
    plt.figure(2)
    for l in acc_list:
        plt.plot(epochs, history.history[l], 'b', label='Training accuracy (' + str(format(history.history[l][-1],'.5f'))+')')
    for l in val_acc_list:    
        plt.plot(epochs, history.history[l], 'g', label='Validation accuracy (' + str(format(history.history[l][-1],'.5f'))+')')

    plt.title('Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.show()

In [ ]:
import matplotlib as plt
from matplotlib import cm

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        title='Normalized confusion matrix'
    else:
        title='Confusion matrix'

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.show()

In [ ]:
#and use helper functions above to show output
#visualization
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from keras.utils import plot_model
#visualize best model
n_classes = 2
model_name = 'dense'
plot_confusion_matrix(confusion_matrix(grid_result.predict_classes(X_test), y_test), n_classes)
plot_history(grid_result.best_estimator_.fit(X_test, y_test, validation_split=0.33, epochs=150, batch_size=10, verbose=0))
#visualize
plot_model(grid_result.best_estimator_, to_file='model_%.png' % model_name)